In [ ]:
#dubbelen eruit
#husselen
#opknippen in delen, 10.000 per stuk?

In [ ]:
import pandas as pd
import os

def process_chunk(dataframes):
    # Combine, deduplicate, and shuffle
    combined = pd.concat(dataframes)
    unique = combined.drop_duplicates()
    shuffled = unique.sample(frac=1).reset_index(drop=True)
    return shuffled

# Assuming all your dataframe names or paths are in a list
all_dataframe_paths = ['path_to_df1', 'path_to_df2', ..., 'path_to_df3000']
chunk_size = 100  # Adjust this based on your system's capability
processed_chunks = []

for i in range(0, len(all_dataframe_paths), chunk_size):
    # Load chunk of dataframes
    chunk_paths = all_dataframe_paths[i:i + chunk_size]
    chunk_dfs = [pd.read_csv(path) for path in chunk_paths]

    # Process and store the chunk
    processed_chunk = process_chunk(chunk_dfs)
    processed_chunk.to_csv(f'processed_chunk_{i // chunk_size}.csv', index=False)
    processed_chunks.append(f'processed_chunk_{i // chunk_size}.csv')

# Finally, combine, shuffle, and batch the processed chunks
final_df = pd.concat([pd.read_csv(chunk) for chunk in processed_chunks])
final_shuffled_df = final_df.sample(frac=1).reset_index(drop=True)

batch_size = 10000
for i in range(0, len(final_shuffled_df), batch_size):
    batch = final_shuffled_df.iloc[i:i + batch_size]
    batch.to_csv(f'final_batch_{i // batch_size}.csv', index=False)

# Optionally, clean up intermediate chunk files
for chunk in processed_chunks:
    os.remove(chunk)


In [ ]:
#verzamel juiste aantal zinnen
#gooi in de api


In [ ]:
import pandas as pd
import langchain
import openai
import re
from environs import Env
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableMap, RunnablePassthrough
from langchain.output_parsers.json import SimpleJsonOutputParser

In [ ]:
env = Env()
env.read_env(".env")  # Read .env file
OPENAI_API_KEY = env("OPENAI_API_KEY")  # Get the API key
OPEN_AI_TOKEN_I_PRICE = (
    0.003 / 1000
)  # Replace X with the current price per token from OpenAI's pricing
OPEN_AI_TOKEN_O_PRICE = 0.006 / 1000
openai.api_key = OPENAI_API_KEY

In [ ]:
def find_indices(sentence, word):
    # check if -1 is needed for IOB notation
    start_index = sentence.find(word)
    end_index = start_index + len(word)
    return [start_index, end_index]


def has_multiple_occurrences(sentence, model_output):
    for category in model_output:
        for word in model_output[category]:
            if sentence.count(word) > 1:
                return True
    return False


def transform_output(sentence, model_output):
    labels = []
    flagged = False

    # If multiple occurences, you should annotate manually
    if has_multiple_occurrences(sentence, model_output):
        return {"Sentence": sentence, "Labels": [], "Flagged": True}

    for category in model_output.keys():
        for word in model_output[category]:
            indices = find_indices(sentence, word)
            if indices:
                labels.append(indices + [category])  # 'animal' or 'plant'

    return {"Sentence": sentence, "Labels": labels, "Flagged": flagged}

In [ ]:
prompt = ChatPromptTemplate.from_template(
    "You'll analyze Old Dutch sentences to identify plants and animals. Given a sentence provide the following fields in a JSON dict: 'plants', 'animals'. Remember: Tag only explicit references to plants or animals. Ignore plant/animal parts, products, and habitats. No tagging of particles. Tag only the nouns that directly refer to the plant or animal, excluding adjectives that are not part of a species' common name or a proper noun. Tag literally (use the exact same spelling as in the Dutch sentence). Text: {x}"
)
model = ChatOpenAI(model="ft:gpt-3.5-turbo-1106:personal::8KmdqIHA")
map_ = RunnableMap(x=RunnablePassthrough())
chain = map_ | prompt | model | SimpleJsonOutputParser()

In [ ]:
def estimate_token_count(text):
    return len(str(text)) / 4


def process_batch(batch):
    input_token_count = 0
    output_token_count = 0
    results = []

    for sentence in batch:
        # Count input tokens
        input_tokens = estimate_token_count(sentence)
        input_token_count += input_tokens

        # API call
        response = chain.invoke(sentence)

        # Count output tokens
        output_tokens = estimate_token_count(response)
        output_token_count += output_tokens

        # Process response
        tagged = transform_output(sentence, response)
        results.append(tagged)

    return results, input_token_count, output_token_count

In [ ]:
results = []
batch_size = 20  # Adjust based on your needs and API limitations
batches = [dutch_lst[i : i + batch_size] for i in range(0, len(dutch_lst), batch_size)]

total_input_tokens = 0
total_output_tokens = 0
for batch in tqdm(batches, desc="Processing batches"):
    batch_results, batch_input_tokens, batch_output_tokens = process_batch(batch)
    results.extend(batch_results)
    total_input_tokens += batch_input_tokens
    total_output_tokens += batch_output_tokens

print(f"Estimated input cost: {total_input_tokens*OPEN_AI_TOKEN_I_PRICE}")
print(f"Estimated output cost: {total_output_tokens*OPEN_AI_TOKEN_O_PRICE}")

# Create and save DataFrame
df = pd.DataFrame(results, columns=["Sentence", "Labels", "Flagged"])
df.to_csv("plants_animals.csv", index=False)

In [ ]:
#zet resultaat van de api om naar IOB

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("emanjacavas/GysBERT")

def align_labels_with_tokens(tokenizer, sentence, labels):
    tokenized_input = tokenizer(sentence, return_offsets_mapping=True)
    token_labels = ["O"] * len(tokenized_input["input_ids"])  # Initialize labels as 'Outside'

    for start, end, label in labels:
        token_start = None
        token_end = None

        # Find the token indices that correspond to the start and end of the span
        for idx, (token_start_pos, token_end_pos) in enumerate(tokenized_input.offset_mapping):
            if token_start is None and token_start_pos >= start:
                token_start = idx
            if token_end_pos <= end:
                token_end = idx

        # Assign labels (using IOB format)
        if token_start is not None and token_end is not None:
            token_labels[token_start] = f"B-{label}"
            for idx in range(token_start + 1, token_end + 1):
                token_labels[idx] = f"I-{label}"

    # Remove the label for special tokens (like [CLS], [SEP])
    token_labels = [label if offset_mapping[0] != offset_mapping[1] else "O" for label, offset_mapping in zip(token_labels, tokenized_input.offset_mapping)]

    return token_labels

# Example usage
sentence = "Kat krabt de krullen van de trap"
labels = [[0, 2, "animal"]]
aligned_labels = align_labels_with_tokens(tokenizer, sentence, labels)
